Desarrollado por María Lourdes Linares Barrera y Pablo Reina Jiménez.   
Proyecto para la asignatura Análisis de Información no Estructurada.  
Máster en Ingeniería del Software Cloud, Datos y Gestión TI.

# 3. Distinguir géneros musicales utilizando modelos de aprendizaje

En esta sección nos centraremos en entrenar un modelo basado en las características obtenidas en el notebook anterior. Con estas características trataremos de identificar si cada una de las canciones pertenecen al género clásico o no clásico.

## Importaciones

In [19]:
import torch
import torchaudio

import numpy as np

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

import os
import netron

____

## 3.1. Modelos de aprendizaje basados en extracción de características (*bag of songs*)

In [20]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001

### 3.1.1. Creación del corpus de datos tabulares de características

En primer lugar, haremos uso de torch para crear un dataset haciendo uso de las características de audio generadas anteiormente. El uso de esta estructura nos facilitará la posterior carga de datos en un DataLoader y su uso para el entrenamiento del modelo. 

In [21]:
class TabularDataset(torch.utils.data.Dataset):

    def __init__(self, features_file, scaler=None):

        df = pd.read_csv(features_file)

        self.X = df.drop(['audio_file', 'label'], axis=1)
        self.y = df['label'].values.astype(np.int64)

        if scaler:
            self.X = scaler.transform(self.X)
        else:
            self.scaler = StandardScaler()
            self.X = self.scaler.fit_transform(self.X)

    def get_scaler(self):
        return self.scaler

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()
    
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)


In [22]:
ccmusic_train = TabularDataset('ccmusic/train/features.csv')
train_scaler = ccmusic_train.get_scaler()
ccmusic_train_dataloader = torch.utils.data.DataLoader(ccmusic_train, 
                                                       batch_size=BATCH_SIZE, 
                                                       shuffle=True)

ccmusic_validation = TabularDataset('ccmusic/validation/features.csv', train_scaler)
ccmusic_validation_dataloader = torch.utils.data.DataLoader(ccmusic_validation, 
                                                            batch_size=BATCH_SIZE, 
                                                            shuffle=False)

ccmusic_test = TabularDataset('ccmusic/test/features.csv', train_scaler)
ccmusic_test_dataloader = torch.utils.data.DataLoader(ccmusic_test, 
                                                      batch_size=BATCH_SIZE, 
                                                      shuffle=False)

### 3.1.2. Definición del modelo para la clasificación de generos en base a las características

En cuanto a la elección del modelo, hemos optado por un modelo simple de perceptrón multicapa, con 3 capas lineales de 128, 64 y 1 neurona respectivamente. El restulado de esta última neurona nos indicará si la predicción corresponde a música clásica o no clásica. Como función de pérdida usaremos BinaryCrossEntropy. Con está función de loss no es necesario aplicar un función de activación sigmoide en la última capa del modelo. Además, haciendo uso del conjunto de validación, hemos aplicado la técnica de early stopping con una paciencia de 5. Con esto nos aseguramos que el modelo no se sobreajuste con los datos de entrenamiento.

In [23]:
class MLPClassifier(torch.nn.Module):

    def __init__(self, input_dim, num_classes):
        super(MLPClassifier, self).__init__()

        self.linear_block = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 128),
            torch.nn.ReLU(),
            # torch.nn.BatchNorm1d(128),
            # torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            # torch.nn.BatchNorm1d(64),
            torch.nn.Linear(64, num_classes if num_classes > 2 else 1)
        )

    def forward(self, x):
        return self.linear_block(x)

In [24]:
def train_single_epoch(model, train_dataloader, val_dataloader, loss_fn, optimizer, device):

    model.train()

    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        predictions = model(inputs)
        if isinstance(loss_fn, torch.nn.BCEWithLogitsLoss):
            loss = loss_fn(predictions, targets.float().unsqueeze(1))
        elif isinstance(loss_fn, torch.nn.CrossEntropyLoss):
            loss = loss_fn(predictions, targets)
        loss.backward()
        optimizer.step()  

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            predictions = model(inputs)
            if isinstance(loss_fn, torch.nn.BCEWithLogitsLoss):
                loss = loss_fn(predictions, targets.float().unsqueeze(1))
            elif isinstance(loss_fn, torch.nn.CrossEntropyLoss):
                loss = loss_fn(predictions, targets)
            val_loss += loss.item()
    val_loss /= len(val_dataloader)

    return loss.item(), val_loss

def train(model, train_dataloader, val_dataloader, loss_fn, optimizer, epochs, patience=5, device='cuda'):

    model.to(device)
    print("Inicio del entrenamiento")

    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        print(f"Época {epoch+1} ", end='')
        loss, val_loss = train_single_epoch(model, train_dataloader, val_dataloader, loss_fn, optimizer, device)
        print(f"Loss: {loss}")

        if loss < best_loss:
            best_loss = loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Deteniendo entrenamiento en la época {epoch+1}")
            break

    print("Fin del entrenamiento")

Una vez entrenado el modelo, pasamos a evaluar los resultados del conjunto de test, para determinar la bondad del modelo. Debido a que el dataset se encuentra desbalanceado, calcularemos las méticas de accuracy y F1, lo que nos indicará si realmente el modelo está realizando buenas predicciones.

In [25]:
def evaluate_model(model, dataloader, num_classes, device='cuda'):
    
    model.eval()
    with torch.no_grad():
        predictions = []
        targets = []
        for inputs, target in dataloader:
            inputs, target = inputs.to(device), target.to(device)
            output = model(inputs)
            if num_classes > 2:
                output = torch.argmax(output, dim=1)
            else:
                output = torch.sigmoid(output)
                output = (output > 0.5)
            predictions.append(output)
            targets.append(target)
        predictions = torch.cat(predictions, dim=0)
        targets = torch.cat(targets, dim=0)
        return {
            'acc': accuracy_score(targets.cpu(), predictions.cpu()),
            'f1': f1_score(targets.cpu(), predictions.cpu()) if num_classes == 2 
            else f1_score(targets.cpu(), predictions.cpu(), average='micro')
        }

### 3.1.3. Entrenamiento e inferencia

Una vez definidas las funciones necesarias procedemos al entrenamiento del modelo. Entrenaremos duarante 50 épocas y una paciencia de 5 para early stopping.

In [26]:
EPOCHS = 50
modelo = MLPClassifier(input_dim=ccmusic_train.X.shape[1], num_classes=2)

Podemos representar la red neuronal antes de entrenar para observar su estructura y comprobar si es necesaria alguna modificación.

In [29]:
print(modelo)

MLPClassifier(
  (linear_block): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [30]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(modelo.parameters(), 
                             lr=LEARNING_RATE)
train(modelo, ccmusic_train_dataloader, ccmusic_validation_dataloader, loss_fn, optimizer, EPOCHS)

Inicio del entrenamiento
Época 1 Loss: 0.08833938091993332
Época 2 Loss: 0.02928599715232849
Época 3 Loss: 0.016496926546096802
Época 4 Loss: 0.009050291031599045
Época 5 Loss: 0.00845313910394907
Época 6 Loss: 0.008805051445960999
Época 7 Loss: 0.0062797898426651955
Época 8 Loss: 0.005658080335706472
Época 9 Loss: 0.0030308288987725973
Época 10 Loss: 0.004965712316334248
Época 11 Loss: 0.003865364706143737
Época 12 Loss: 0.0034403956960886717
Época 13 Loss: 0.0019635818898677826
Época 14 Loss: 0.002340928418561816
Época 15 Loss: 0.001330442144535482
Época 16 Loss: 0.0012262616073712707
Época 17 Loss: 0.0015109622618183494
Época 18 Loss: 0.0005528116598725319
Época 19 Loss: 0.0010406887158751488
Época 20 Loss: 0.0004564994596876204
Época 21 Loss: 0.00037578705814667046
Época 22 Loss: 0.0003343092685099691
Época 23 Loss: 0.00034876231802627444
Época 24 Loss: 0.0002993019297719002
Época 25 Loss: 0.00022636445646639913
Época 26 Loss: 0.00020564223814290017
Época 27 Loss: 0.000197106026462

Una vez entrenado evaluamos sobre el conjunto de test.

In [31]:
metrics_test = evaluate_model(modelo, ccmusic_test_dataloader, num_classes=2)
print(f"Accuracy en el conjunto de test: {metrics_test['acc']}")
print(f"F1 en el conjunto de test: {metrics_test['f1']}")

Accuracy en el conjunto de test: 0.9651162790697675
F1 en el conjunto de test: 0.9777777777777777


Vemos que las métricas sobre test son muy buenas. Estos nos indica que, efectivamente, las características obtenidas en los pasos anteriores identifican a cada uno de los dos géneros musicales de forma efectiva. Además, cabe destacar, que estos resultados se obtienen con un conjunto de datos y modelo simples, lo que permite su entrenamiento en cualquier máquina sin ningún requisito específico.

___